In [1]:
import lightgbm as lgb
import numpy as np
import pandas as pd
import hashlib

For now, we'll combine the lexical-only features and those with Bowen's features

In [2]:
lexical = pd.read_csv("lexical_only.csv").dropna()
lexical_bowen = pd.read_csv("with_bowen.csv").dropna()


lexical_features = []

# Add hashes for lexical features
lexical = lexical.rename(
    columns={
        "Unnamed: 0": "experiment_name",
        "rouge>0.4": "rouge",
        "bertscore>0.8": "bertscore",
        "cosine_sim>0.8": "cosine_sim",
        "entity_sim>0.8": "entity_sim",
        "bertscore_length>0.4": "bertscore_length",
    }
)
lexical["hash"] = lexical["experiment_name"].apply(
    lambda x: hashlib.md5(
        x.split("FEATS")[-1].removeprefix("_").encode("utf-8")
    ).hexdigest(),
)
lexical_feats = lexical[
    [
        "hash",
        "rouge",
        "bertscore",
        "cosine_sim",
        "entity_sim",
        "bertscore_length",
        "label",
        "Overall",
    ]
]

lexical_bowen = lexical_bowen[
    ["hash"]
    + ["rouge", "bertscore", "cosine_sim", "entity_sim", "bertscore_length"]
    + [col for col in lexical_bowen.columns if "=" in col]
    + ["label", "Overall"]
]


df = pd.concat([lexical_feats, lexical_bowen]).fillna(0)

In [3]:
df

,hash,rouge,bertscore,cosine_sim,entity_sim,bertscore_length,label,Overall,complexity_of_intents=complex,complexity_of_intents=moderate,...,subject_of_expertise=Political_science,subject_of_expertise=Psychology,subject_of_expertise=Public_administration,subject_of_expertise=Religion,subject_of_expertise=Social_work,subject_of_expertise=Sociology,subject_of_expertise=Space_sciences,subject_of_expertise=System_science,subject_of_expertise=Visual_arts,type_of_in_context_material=1
0,a9a4c59b129c0a6816b12b6ca883e049,0.0,1.0,0.0,0.0,1.0,1.0,0.757970,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,52009160febbb60893944fedd6d927fc,0.0,1.0,1.0,0.0,0.0,1.0,0.735185,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,8275988da88b1aae55e874392fe1cb30,1.0,0.0,0.0,0.0,1.0,1.0,0.732124,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,d8b9477149b0d921769c9e2041155645,0.0,0.0,0.0,0.0,1.0,1.0,0.731826,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,d809a5a3c85aaf3e7fc43f648361f31b,1.0,0.0,0.0,0.0,1.0,1.0,0.730485,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,01414f78b534bd2b158581a9e90bb978,0.0,0.0,0.0,0.0,0.0,0.0,0.656139,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
118,7c8418ae17d6ca4b5c5c332685d31241,0.0,0.0,0.0,0.0,0.0,0.0,0.655546,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
119,78875c3aa32dbc58dbdd46096a9c877e,0.0,0.0,0.0,0.0,0.0,0.0,0.655163,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
120,e01c9cf683de4b21fe2546acc5eefb64,0.0,0.0,0.0,0.0,0.0,0.0,0.648629,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
